In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import string
from bert_serving.client import BertClient
bc = BertClient()
data=pd.read_csv("/Users/prakash/Desktop/natural-to-sql/Dataset/Augmented.csv")
X_train, X_test, y_train, y_test = train_test_split(data["Natural"],data["SQL"], test_size=0.2, random_state=42)

In [2]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/prakash/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
arr=["AffiliationID_Place_Affiliation.csv","AuthID_AffiliationID.csv","AuthID_FieldID.csv","AuthID_Name.csv"
     ,"ConfID_FieldID.csv","ConfID_PaperID.csv","ConfID_Venue_Year.csv","FieldID_Topic.csv","KeywordID_PaperID.csv"
     ,"PaperID_AuthID.csv","PaperID_FieldID.csv","PaperID_Summary.csv","PaperID_Title.csv"]

maping={}
table={}
for i in range(13):
    data=pd.read_csv("/Users/prakash/Desktop/natural-to-sql/Dataset/"+arr[i])
    table[arr[i]]=data.columns
    for val in data.columns:
        if(val in maping):
            maping[val]=list(set(data[val]).intersection(set(maping[val])))
        else:
            maping[val]=list(set(data[val]))

In [4]:
from difflib import get_close_matches 
# r=0
embed=[]
place_holder=[]
for i in X_train.keys():
    if(X_train[i][-1]=='.' or X_train[i][-1]=='?'):
        arr=X_train[i][:-1].strip().split(" ")
    else:
        arr=X_train[i].strip().split(" ")
    temp=[]
    for j in range(len(arr)):
        if(arr[j] not in stop):
            temp.append(arr[j])
    close=[]
#     print(i,X_train[i],temp)
    temp1=[]
    for val in temp:
        y=get_close_matches(val, [ele for ele in maping])
        if(len(y)>0):
            close.extend(y)
            temp1.append("$col$")
        else:
            temp1.append(val)
    match=[]
    temp2=[]
    for val in temp1:
        temp2.append(val.lower())
        for ele in maping:
            if(val in maping[ele]):
                match.append(val)
                temp2.pop()
                temp2.append("$val$")
                if(ele not in close):
                    close.append(ele)
#     print(temp2) 
#     print(close,match) 
#     for val in table:
#     r+=1
    gio=bc.encode([" ".join(temp2)])
    place_holder.append(temp2)
    print(len(gio.flatten()))
    embed.append(gio.flatten())
#     if(r>10):
#         break
print(len(embed),len(place_holder))
#Contains begin and ends latter

768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768
768


In [10]:
#check almost match with the column names and fix the columns and table
#check all possiblity then revert back with reward for each answer
train_pre={"embed":embed,"place_holder":place_holder}
#Get embedding and replace the values
X_train.to_csv("X_train.csv",index=False)
X_test.to_csv("X_test.csv",index=False)
y_train.to_csv("Y_train.csv",index=False)
y_test.to_csv("Y_test.csv",index=False)
t=pd.DataFrame(train_pre)
t.to_csv("embed.csv",index=False)

In [11]:
r=0
embed=[]
place_holder=[]
for i in X_test.keys():
    if(X_test[i][-1]=='.' or X_test[i][-1]=='?'):
        arr=X_test[i][:-1].split(" ")
    else:
        arr=X_test[i].split(" ")
    temp=[]
    for j in range(len(arr)):
        if(arr[j] not in stop):
            temp.append(arr[j])
    close=[]
#     print(i,X_train[i],temp)
    temp1=[]
    for val in temp:
        y=get_close_matches(val, [ele for ele in maping])
        if(len(y)>0):
            close.extend(y)
            temp1.append("$col$")
        else:
            temp1.append(val)
    match=[]
    temp2=[]
    for val in temp1:
        temp2.append(val.lower())
        for ele in maping:
            if(val in maping[ele]):
                match.append(val)
                temp2.pop()
                temp2.append("$val$")
                if(ele not in close):
                    close.append(ele)
#     print(temp2) 
#     print(close,match) 
#     for val in table:
    r+=1
    gio=bc.encode([" ".join(temp2)])
    place_holder.append(temp2)
#     print(len(gio.flatten()))
    embed.append(gio.flatten())
    if(r>10):
        break
test_pre={"embed":embed,"place_holder":place_holder}

In [12]:
for i in range(len(test_pre["embed"])):
    val=test_pre["embed"][i]
    temp=0
    for j in range(len(train_pre['embed'])):
        com=train_pre["embed"][j]
        t=sum([val[k]*com[k] for k in range(len(val))])/(sum([val[k]**2 for k in range(len(val))])*sum([com[k]**2 for k in range(len(val))]))**(1/2)
        if(t>temp):
            temp=t
            max_val=j
    print(test_pre["place_holder"][i],temp,train_pre["place_holder"][max_val])

['$col$', '$col$', '$col$', '$val$', ''] 1.0000000047060085 ['$col$', '$col$', '$col$', '$val$', '']
['what', '$col$', 'id', '$col$', '$col$', 'learning', 'transliteration', 'lexicons', 'from', 'the', 'web'] 1.0000000020859952 ['what', '$col$', 'id', '$col$', '$col$', 'learning', 'transliteration', 'lexicons', 'from', 'the', 'web']
['what', '$col$', 'id', '$col$', '$col$', 'improved', 'iterative', 'correction', 'distant', 'spelling', 'errors'] 0.9999999868621529 ['what', '$col$', 'id', '$col$', '$col$', 'improved', 'iterative', 'correction', 'distant', 'spelling', 'errors']
['what', '$col$', '$col$', '$col$', '$val$'] 1.00000001169172 ['what', '$col$', '$col$', '$col$', '$val$']
['which', 'conference', 'held', '$val$', '$col$'] 1.000000019068424 ['which', 'conference', 'held', '$val$', '$col$']
['how', 'many', '$col$', 'written', '$val$'] 1.0000000167686076 ['how', 'many', '$col$', 'written', '$val$']
['list', '$col$', 'id', '$col$', 'conference', 'id', '$val$'] 1.0000000104623892 ['li

In [13]:
for i in y_test.keys():
    temp4=y_test[i].strip().split()
    print(temp4[1],temp4[3])
    break

Summary PaperID_Summary
